# 导入模块

In [185]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False
import json
import warnings
warnings.filterwarnings("ignore")
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [186]:
clean = Data_Clean()

In [187]:
f_path="F:/myfile/p站数据/当月数据"
#f_path="F:/下班前订单数据"
#f_path="F:/订单日报/临时导出"

In [188]:
import os
excel_names = []
excel_date = []
for excel_name in os.listdir(f_path):
    # excel_names.append(excel_name)
    # excel_date.append(excel_name.split('.')[0])
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        excel_names.append(excel_name)
        excel_date.append(excel_name.split('.')[0])
print(excel_names)
print(excel_date)


['0901-0915.xlsx']
['0901-0915']


In [189]:
# df_list=[]
# for excel_name in excel_names:
#     # 读取每个excel到df
#     excel_path = f"{f_path}/{excel_name}"
#     print(excel_path)
#     df_split = pd.read_excel(excel_path,skiprows=0)
#     print(df_split)
#     df_list.append(df_split)
#     print(df_list)

# df = pd.concat(df_list,ignore_index=True)

In [190]:
df = pd.read_excel(f'F:/myfile/p站数据/当月数据/{excel_date[-1]}.xlsx')

In [191]:
df["下单日期"]=df["下单时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month

In [192]:
df = df[df['商品属性'].notnull()]

# 信息获取

In [193]:
df["拒绝理由"]=df["备注信息"].str.split("拒绝理由：").str[1]
df["拒绝理由"]=df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')
df["取消原因"]=df["备注信息"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["备注信息"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
#df["无法联系原因"]=df["备注信息"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["下单商品"].str.contains('99新')|df["下单商品"].str.contains('95新')|df["下单商品"].str.contains('准新'),"二手","全新")
df.loc[:,"租赁方案"]=np.where(df["商品属性"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["预授权占用自有资金"]>0,"部分免押","全免押")
df.loc[:,"是否使用优惠券"]=np.where((df["设置买断金额"]>df["总租金"])&(df["租赁方案"]=="租完即送"),"使用优惠券","未使用优惠券")
df.loc[:,"是否进行预授权"]=np.where(df["预授权占用自有资金"].isnull(),"未预授权","已预授权")

In [194]:
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["商品属性"]),axis=1)

In [195]:
def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["商品属性"]),axis=1)

In [196]:
df.loc[:,"进件"]=np.where((df["订单状态"]=="待支付")|(df["订单状态"]=="订单取消"),"未进件","进件")

# 渠道归属

In [197]:
df.loc[:,"来源渠道"]=df["来源渠道"].fillna("未知渠道")

In [198]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if '京东活动' in b:
        return '京享租'
    elif "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "支付宝直播" in a or '支付宝直播' in b or '支付宝直播商品' in b:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in b:
        return "生活号"
    elif "社群" in b or '支群' in b:
        return "支付宝社群"
    
    else :
        return a

In [199]:
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["活动名称"]),axis=1)

In [200]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["订单状态"].map(dict_status_code)

In [201]:
df.sort_values(by=["下单日期","状态编码"],inplace=True)

# 删除重复数据

In [202]:
#删除重复单号
df.drop_duplicates(subset=["单号"],inplace=True)
#删除身份证空值行
df.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['活动名称']=="线下门店3个月试行"].index,inplace=True)
#删除订单状态空值行
df.dropna(subset=["订单状态"],axis=0,inplace=True)
df.drop(df[df['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)

In [203]:
#删除重复订单
df.drop_duplicates(subset=["单号"],inplace=True)
df.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)

# 定义状态

In [204]:
def reject_type(a,b,c,d,e,f,g):
    if a is not np.nan and b=="未进件":
        return "前置拦截"
    elif g is np.nan and b=="未进件":
        return "进件前取消"
    elif a is not np.nan and b=="进件":
        return "机审强拒"
    elif c is not np.nan and"抖音非免押用户" in c:
        return "抖音非免押拒绝"
    elif a is np.nan and b=="进件" and c is not np.nan and '已退款' in e:
        return "人审拒绝"
    elif a is np.nan and b=="进件" and d is not np.nan:
        return "客户取消"
    #elif a is np.nan and b=="进件" and f is not np.nan:
        #return "无法联系"
    elif g is not np.nan and b=="进件" and "蚂蚁数控风险等级" in g:
        return "出库前风控强拒"
    elif g is not np.nan and b=="进件" and "命中出库前风控流强拒" in g:
        return "出库前风控强拒"
    elif a is np.nan and b=="进件" and "待审核" in e:
        return "待审核"
    elif a is np.nan and b=="进件" and "待发货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "待收货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "租赁中" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    else :
        return e
    
    

In [205]:
df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["订单状态"],x["订单状态"],x["备注信息"]),axis=1)

In [206]:

fw_path = 'F:/myfile/p站数据/服务订单'
fw_excel_names = []
fw_excel_date = []
for excel_name in os.listdir(fw_path):
    fw_excel_names.append(excel_name)
    fw_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(fw_excel_date)


['~$服务订单0611-0617', '服务订单0901-0915']


In [207]:
f_path_insure=f"F:/myfile/p站数据/服务订单/{fw_excel_date[-1]}.xlsx"
df_insure=pd.read_excel(f_path_insure,skiprows=0)
df_insure1=pd.merge(df,df_insure,left_on="单号", right_on="租赁订单号",how="left")
df=df_insure1

In [208]:
import datetime

Today=datetime.date.today()
str(Today)
Now=str(datetime.datetime.now() )
Now

'2025-09-17 14:36:53.686581'

# 删除商家数据

In [209]:
df_contain = df.copy()
# df_contain[df_contain['商家']=='星晟数码'].shape
# 选取商家为人人享租，喜卓灵新租机的订单数据，宏生需求，2025-09-11
# df_ = df[df['商家'].isin(['人人享租','喜卓灵新租机'])]
# df_.商家.unique()

## 获取出库类台账数据（宏生）

In [210]:
# import pymysql
# def query(sql,
#         host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
#         user="ylc",
#         password="1O8t5lcJ5aMhwwPEUUjS",
#         database = '' ,
#         port=3306
#         ):   
#     conn = pymysql.connect(
#         host=host,
#         user=user,
#         port=port,
#         password=password,
#         max_allowed_packet=1073741824,
#         charset="utf8")
#     try:
#         df = pd.read_sql(sql, con=conn)
#         conn.close()
#     except:
#         print('error')
#         conn.close()
#         raise
#     return df

In [211]:
# # 订单号  租金om.all_money	已付租金tos.real_pay_money(tos.status=3)	期数tos.sort  碎屏险（是否有碎屏险）buy_service_product，1否2是

# sql_ck = '''
# select om.id order_id,om.order_number 订单号, om.all_money, tos.real_pay_money, tos.sort, om.buy_service_product,om.status

# from db_digua_business.t_order om
# left join db_digua_business.t_order_stages tos on om.id=tos.order_id
# where om.user_mobile is not null 
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-08-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-10'
# and tos.status=3
# '''
# df_ck = query(sql_ck)

# df_ck.shape

## 去重

In [212]:
# # 按照order_id、sort去重
# duplicates = df_ck[df_ck.duplicated(subset=['order_id', 'sort'], keep=False)]
# if not duplicates.empty:
#     print("发现重复数据，数量：", len(duplicates))
#     # 可以选择保留最新的记录（根据实际业务逻辑调整）
#     # 这里假设reality_refund_date不为空的是更新的记录
#     df_ck = df_ck.sort_values(['order_id', 'sort', 'refund_date'], ascending=[True, True, False])
#     # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
#     df_ck = df_ck.drop_duplicates(subset=['order_id', 'sort'], keep='first')
    
# df_ck["是否出库"] = np.where(df_ck["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
# df_ck["是否有碎屏险"] = np.where(df_ck["buy_service_product"] == 2, "有碎屏险", "无碎屏险")
# df_ck.loc[:,"已付租金"]=df_ck["real_pay_money"].fillna(0)
# df_ck.loc[:,"期数"] = df_ck["sort"]
# df_ck.loc[:,"总租金"]=df_ck["all_money"]
# df_ck = df_ck[df_ck['是否出库'] == 1][['order_id', '订单号', '总租金', '已付租金', '期数', '是否有碎屏险']]
# df_ck.columns

In [213]:

# dfck=pd.merge(df_ck,df_,left_on="订单号", right_on="单号")
# dfck.shape


In [214]:

df.drop(df[df['商家']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="优优2店"].index,inplace=True)
df.drop(df[df['商家']=="探路者自营账户"].index,inplace=True)
df.drop(df[df['商家']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['商家']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['商家']=="北京海鸟窝科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="澄心优租"].index,inplace=True)
df.drop(df[df['商品型号'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
# 趣智数码  单
df.drop(df[df['商家']=="趣智数码"].index,inplace=True)
df.drop(df[df['商家']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['商家']=="广州康基贸易有限公司"].index,inplace=True)
df.drop(df[df['商家']=="汇客好租"].index,inplace=True)
df.drop(df[df['商家']=="线下小店"].index,inplace=True)
df.drop(df[df['商家']=="乙辉数码"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="南京聚格网络科技"].index,inplace=True)
df.drop(df[df['商家']=="呱子笔记本电脑"].index,inplace=True)

df.drop(df[df['商家'] == "星晟数码"].index, inplace=True)
df.drop(df[df['商家'] == "蘑菇时间"].index, inplace=True)
# df = clean.drop_merchant(df)

In [215]:
df_j=df[df["进件"]=="进件"]

In [216]:
# 商家拒量
df.drop(df[df['商家']=="小蚂蚁租机"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="人人享租"].index,inplace=True)
df.drop(df[df['商家']=="崇胜数码"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵租机"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵新租机"].index,inplace=True)

In [217]:
df_j[df_j.归属渠道=='搜索渠道'].shape

(5641, 88)

# 以下为出库相关数据，要统计到出库相关的数据，必须运行到以下读取出库台账的代码

In [218]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023",skiprows=0) 
f_path_ck="F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck=pd.read_excel(f_path_ck,sheet_name="2025",skiprows=0)
dfck=pd.merge(df_ck,df,left_on="订单号", right_on="单号")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape


(1977, 106)

In [219]:
dfck.订单状态.value_counts()

订单状态
租赁中    1546
待收货     229
待发货     114
已完成      40
已退款      32
待审核      16
Name: count, dtype: int64

# 检查出库订单中有没有客户取消的单，即订单状态为”已退款"的单，找出来删除后重新运行上一块导入出库台账的代码

In [220]:
dfck.drop(dfck[dfck["订单状态"]=="已退款"].index,inplace=True)

# 剔除拒收!!!!
dfck.drop(dfck[dfck["订单状态"]=="已完成"].index,inplace=True)

In [221]:
dfck.订单状态.value_counts()

订单状态
租赁中    1546
待收货     229
待发货     114
待审核      16
Name: count, dtype: int64

In [222]:
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)

# 苹果15系列商品型号-进件

In [223]:
# #苹果15系列机型进件
# df_j["型号内存"]=df_j["商品型号"]+"_"+df_j["内存"]
# df_jap15=df_j[df_j["型号内存"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["型号内存"],margins=True).to_excel(f"F:苹果15进件.xlsx")

In [224]:
# #华为进件
# df_jhw=df_j[df_j["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# pd.crosstab(df_jhw["商品型号"],df_jhw["内存"],margins=True)


In [225]:
# df_jhw.groupby(["商品型号","内存","颜色"]).agg({"单号":np.size}).unstack(-1)

# 苹果15系列商品型号-出库

In [226]:
# # 苹果15系列商品型号-出库
# df_ap15=dfck[dfck["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True)

In [227]:
# #华为出库
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]

# pd.crosstab(dfckhw["商品型号"],dfckhw["内存_x"],margins=True)

In [228]:
# dfckhw.groupby(["商品型号","内存_y","颜色_x"]).agg({"订单号":np.size}).unstack(-1)

# 某渠道苹果15系列商品型号-进件

In [229]:
#苹果15系列商品型号-去重订单
# df_qd=df_j[df_j["来源渠道"]=="芝麻租物"]
# df_jap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["商品型号"],margins=True).to_excel("F:/芝麻租物/2024.3芝麻渠道15进件.xlsx")

# 某渠道苹果15系列商品型号-去重订单


In [230]:
# # 某渠道苹果15系列商品型号-进件
# df_qd=df[df["来源渠道"]=="芝麻租物"]
# df_ap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15去重订单.xlsx")

# 某渠道苹果15系列商品型号-出库

In [231]:
# # 某渠道苹果15系列商品型号-出库
# dfck_qd=dfck[dfck["来源渠道"]=="芝麻租物"]
# dfck_jap15=dfck_qd[dfck_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(dfck_jap15["下单日期"],dfck_jap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15出库.xlsx")

# 宏生碎屏险数据，导出来直接甩给他就可以了

In [232]:
# #总体出库订单碎屏险购买数据
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
# df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"] #+df_s2merge["待支付"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库"]/df_s2merge["合计出库"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
# df_s2merge

In [233]:
# #芝麻租物出库订单碎屏险购买数据
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]+df_zws2merge["待确认"]#+df_s2merge["待支付"]
# df_zws2merge

In [234]:
# #Mate60Pro 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60Pro",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]#+df_zws2merge["待确认"]
# df_zws2merge

In [235]:
# #Mate60 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# # df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]   # df_zws2merge["待确认"]+
# df_zws2merge

# 总体碎屏险数据

In [236]:
# #总体碎屏险数据,宏生需求
# df_bx=pd.crosstab(df["下单日期"],df["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx=df_bx.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单",})
# df_bx_j=pd.crosstab(df_j["下单日期"],df_j["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_j=df_bx_j.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件",})
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_spx1=pd.merge(df_bx,df_bx_j,left_index=True, right_index=True)
# df_spx2=pd.merge(df_spx1,df_bx_ck,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/总体碎屏险数据{excel_date[-1]}.xlsx")

In [237]:
# df_s1=pd.crosstab(df["下单日期"],df["服务订单状态"],margins=True)
# df_s1=df_s1.rename(columns={"All":"合计购买碎屏险订单"})
# df_s2=pd.crosstab(df_j["下单日期"],df_j["服务订单状态"],margins=True)
# df_s2=df_s2.rename(columns={"All":"合计购买碎屏险进件"})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.rename(columns={"All":"合计购买碎屏险出库"})
# df_s1merge=pd.merge(df_s1,df_s2,left_index=True, right_index=True)
# df_s2merge=pd.merge(df_s1merge,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"]
# df_s2merge

# 芝麻租物碎屏险数据

In [238]:
# #芝麻租物碎屏险数据,宏生需求
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zw1=pd.crosstab(dfzw["下单日期"],dfzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw1=df_zw1.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单"})

# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zw2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw2=df_zw2.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件"})
# df2=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(df2["下单日期"],df2["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zwspx1=pd.merge(df_zw1,df_zw2,left_index=True, right_index=True)
# df_zwspx2=pd.merge(df_zwspx1,df_zw3,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/芝麻租物碎屏险数据{excel_date[-1]}.xlsx")

In [239]:
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zws1=pd.crosstab(dfzw["下单日期"],dfzw["服务订单状态"],margins=True)
# df_zws1=df_zws1.rename(columns={"All":"合计购买碎屏险订单"})
# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zws2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["服务订单状态"],margins=True)
# df_zws2=df_zws2.rename(columns={"All":"合计购买碎屏险进件"})
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.rename(columns={"All":"合计购买碎屏险出库"})
# df_zws1merge=pd.merge(df_zws1,df_zws2,left_index=True, right_index=True)
# df_zws2merge=pd.merge(df_zws1merge,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=+df_zws2merge["服务中"]+df_zws2merge["待确认"]
# df_zws2merge

In [240]:
#dfck[dfck["是否购买服务商品：1、否；2、是"]==2].to_excel(f"C:/Users/Administrator/Desktop/总体碎屏险数据出库_1119.xlsx")

# 月中\月底运营数据复盘 

# 月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比

In [241]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,来源渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,9,京享租右卡,6,7274.735000,12038.700000,9332.333333,77.95%
1,9,付费流量(通过商家数字推广平台，灯火等投放的广告),2,8065.050000,14296.500000,10999.000000,73.33%
2,9,八派信息,8,13321.650000,13321.650000,10249.000000,129.98%
3,9,公域搜索带券,1,7331.880000,12996.700000,9999.000000,73.33%
4,9,其他渠道场景渠道。,4,8381.507500,12346.800000,9499.000000,88.24%
5,9,小程序商家消息（服务提醒）,1,12996.700000,12996.700000,9999.000000,129.98%
6,9,我的小程序入口,3,4314.526667,8447.400000,6499.000000,66.39%
7,9,搜索-租手机,42,7422.820238,12749.119048,9808.523810,75.68%
8,9,搜索-租苹果手机,18,7768.790556,13213.333333,10165.666667,76.42%
9,9,搜索下挂-商品,135,7323.499481,13112.237778,10087.888889,72.60%


In [242]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,归属渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,9,京享租,469,7792.356994,12680.633902,9842.923241,79.17%
1,9,付费流量(通过商家数字推广平台，灯火等投放的广告),2,8065.050000,14296.500000,10999.000000,73.33%
2,9,八派信息,8,13321.650000,13321.650000,10249.000000,129.98%
3,9,其他渠道场景渠道。,4,8381.507500,12346.800000,9499.000000,88.24%
4,9,小程序商家消息（服务提醒）,1,12996.700000,12996.700000,9999.000000,129.98%
5,9,我的小程序入口,2,3482.765000,6172.750000,4749.000000,73.34%
6,9,搜索渠道,566,7442.869841,12954.730389,9969.318021,74.66%
7,9,支付宝客户端首页,13,7293.870000,11286.961538,8683.615385,84.00%
8,9,支付宝直播,4,6498.955000,13646.600000,10499.000000,61.90%
9,9,支群-社群专享,1,7331.880000,12996.700000,9999.000000,73.33%


# 月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比

In [243]:
dfck_all=dfck.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,9,1905,6889.055218,12814.03748,9899.349186,9899.349186,69.59%


In [244]:
#月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比
dfck_fjd = dfck[dfck.归属渠道!='京享租']
dfck_all=dfck_fjd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,9,1436,6594.035348,12857.607312,9917.777994,9917.777994,66.49%


# 京享租

In [245]:
dfck_jd = dfck[dfck.归属渠道=='京享租']
dfck_all=dfck_jd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,9,469,7792.356994,12680.633902,9842.923241,9842.923241,79.17%


# 月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比

In [246]:
#月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,9,租完即送,31,12615.148387,12615.148387,9705.451613,9705.451613,129.98%
1,9,租完归还,1405,6461.185167,12862.956940,9922.462776,9922.462776,65.12%


# 京享租

In [247]:
dfck_all=dfck_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,9,租完归还,469,7792.356994,12680.633902,9842.923241,9842.923241,79.17%


# 月中月底各渠道各租赁方案出库数

In [248]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["来源渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
来源渠道,,,
京享租右卡,0,6,6
付费流量(通过商家数字推广平台，灯火等投放的广告),0,2,2
八派信息,8,0,8
公域搜索带券,0,1,1
其他渠道场景渠道。,1,3,4
小程序商家消息（服务提醒）,1,0,1
我的小程序入口,0,3,3
搜索-租手机,2,40,42
搜索-租苹果手机,1,17,18


In [249]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["归属渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
归属渠道,,,
京享租,0,469,469
付费流量(通过商家数字推广平台，灯火等投放的广告),0,2,2
八派信息,8,0,8
其他渠道场景渠道。,1,3,4
小程序商家消息（服务提醒）,1,0,1
我的小程序入口,0,2,2
搜索渠道,16,550,566
支付宝客户端首页,3,10,13
支付宝直播,0,4,4


# 月中月底各渠道租赁方案进件数

In [250]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
来源渠道,,,
WIFI6专属,2,0,2
九州信息,3,1,4
京享租右卡,0,348,348
付费流量(通过商家数字推广平台，灯火等投放的广告),4,31,35
八派信息,648,50,698
公域搜索带券,0,1,1
其他渠道场景渠道。,14,119,133
单人聊天会话中的小程序消息卡片（分享）,15,971,986
号卡验收,1,0,1


In [251]:
df_zulin = pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)
# df_zulin.to_excel(f'F:/各渠道租赁进件数/{excel_date[-1]}进件数.xlsx')

In [252]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["归属渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
归属渠道,,,
WIFI6专属,2,0,2
九州信息,3,1,4
京享租,0,9950,9950
付费流量(通过商家数字推广平台，灯火等投放的广告),4,28,32
八派信息,648,47,695
其他渠道场景渠道。,14,90,104
单人聊天会话中的小程序消息卡片（分享）,15,950,965
号卡验收,1,0,1
宜品花,37,1,38


# 以下代码为测算毛利时候使用，使用前先从M站系统导出出库单获取成本数据，然后按顺序运行以下代码

In [253]:

ck_path = 'F:/myfile/p站数据/出库单'
ck_excel_names = []
ck_excel_date = []
for excel_name in os.listdir(ck_path):
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        ck_excel_names.append(excel_name)
        ck_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(ck_excel_date)
os.listdir(ck_path)

['9月出库']


['9月出库.xlsx', '~$7月出库.xlsx']

In [255]:
f_path_cost=f"F:/myfile/p站数据/出库单/{ck_excel_date[-1]}.xlsx"
df_cost=pd.read_excel(f_path_cost,sheet_name='出库表',skiprows=0)
#df_cost.loc[:,"成本"]=np.where(df_cost["供应商"].str.contains('侯立')|df_cost["供应商"].str.contains('张陵')|df_cost["供应商"].str.contains('拾贝壳')|df_cost["供应商"].str.contains('大龙发'),(df_cost["设备成本"]-10)/1.025,df_cost["设备成本"])
# df_cost["成本"]=df_cost["设备成本"]
# df_cost["成本"]=df_cost["成本"].round()
dfck_cost=pd.merge(dfck,df_cost,left_on="订单号", right_on="订单号")
# 更改原因：由于台账存在成本字段与出库单表的成本字段发生冲突
dfck_cost.loc[:, "成本"]=np.where(dfck_cost["设备成本"] is np.nan, dfck_cost["成本"],dfck_cost["设备成本"])
dfck_cost["成本"]=dfck_cost["成本"].round()
dfck_cost.drop_duplicates(subset=["订单号"],inplace=True)
# 宏生需求
# dfck_cost.to_excel(f"F:/毛利/明细_{Today}.xlsx")

In [161]:
dfck_cost.columns

Index(['日期', '订单号', '类型', '型号', '内存_x', '颜色_x', '平台套餐', '租金', '已付租金', '期数',
       ...
       '仓库名称', '设备成本', '设备零售价', '成新度', '设备串码_y', 'SN码', '出库原因', '操作人', '出库数量',
       '出库时间'],
      dtype='object', length=121)

In [162]:
# 毛利计算过程，这块代码不能忽略
dfck_cost.loc[:,"原始毛利"]=dfck_cost["买断金额"]-dfck_cost["成本"]+dfck_cost["商品溢价费"]
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),"苹果","非苹果")
dfck_cost["商品型号_all"] = dfck_cost["商品型号"][dfck_cost["商品型号"].str.contains('iP')==True] 
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),  
#                                 np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16", "苹果15"),"非苹果")
dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),  
                                np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16",
                                        np.where(dfck_cost['商品型号_all'].str.contains('15'), "苹果15",
                                                np.where(dfck_cost['商品型号_all'].str.contains('14'), "苹果14", '苹果14以下'))),"非苹果")
def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果16":
        return s["成本"]*0.6
    elif s["残值机型"]=="苹果15":
        return s["成本"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["成本"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["成本"]*0.375
    else:
        return s["成本"]*0.425
dfck_cost.loc[:,"残值"]=dfck_cost.apply(getrecanzhi,axis=1)
dfck_cost.loc[:,"总应收"]=dfck_cost["总租金"]+dfck_cost["残值"]+dfck_cost["商品溢价费"]
dfck_cost.loc[:,"残值估算毛利"]=dfck_cost["总应收"]-dfck_cost["成本"]
def getremark(s):
    if s["成本"]==0:
        return s["已付金额"]
    else:
        return s["残值估算毛利"]
dfck_cost["残值估算毛利"]=dfck_cost.apply(getremark,axis=1)
dfck2 = dfck_cost[dfck_cost["成本"].notnull()]

In [163]:
# dfck_cost.groupby('月份').agg({'订单号': 'count', '总租金': 'sum', '残值': 'sum', '商品溢价费': 'sum', '总应收': 'sum', '成本': 'sum', '残值估算毛利': 'mean', '买断金额': 'sum', '原始毛利': 'mean'})

# 月中、月底总体数据测算毛利

In [164]:
# 月中、月底总体数据测算毛利
dfck2_fjd = dfck2[dfck2.归属渠道!='京享租']
dfck_zsb=dfck2_fjd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,1327,8816.460437,12843.455087,4026.99465,2938.779706,33.33%,45.68%


In [165]:
dfck2_jd = dfck2[dfck2.归属渠道=='京享租']
dfck_zsb=dfck2_jd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,448,8755.154018,12674.73125,3919.577232,4270.611473,48.78%,44.77%


# 租物和非租物

In [166]:
# # 月中、月底总体数据测算毛利
# dfck2_zw = dfck2[dfck2.归属渠道=='芝麻租物']
# dfck_zsb=dfck2_zw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb


In [167]:
# dfck2_fzw = dfck2[dfck2.归属渠道!='芝麻租物']
# dfck_zsb=dfck2_fzw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb

# 月中、月底分租赁方案测算毛利

In [168]:
dfck_zsb=dfck2.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,租完即送,28,8618.214286,12574.267857,3956.053571,3956.053571,45.90%,45.90%
1,9,租完归还,1747,8803.916428,12804.502003,4000.585575,3264.009794,37.07%,45.44%


In [169]:
#月中、月底分租赁方案测算毛利
dfck_zsb=dfck2_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,租完即送,28,8618.214286,12574.267857,3956.053571,3956.053571,45.90%,45.90%
1,9,租完归还,1299,8820.733641,12849.257429,4028.523788,2916.852325,33.07%,45.67%


# 京享租

In [170]:
dfck_zsb=dfck2_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,租完归还,448,8755.154018,12674.73125,3919.577232,4270.611473,48.78%,44.77%


# 月中、月底分渠道测算毛利

In [171]:
# 月中、月底分渠道测算毛利
dfck_zsb=dfck2.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,来源渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,京享租右卡,6,8315.0,12038.700000,3723.7,3948.735000,47.49%,44.78%
1,9,付费流量(通过商家数字推广平台，灯火等投放的广告),2,9820.0,14296.500000,4476.5,4137.050000,42.13%,45.59%
2,9,八派信息,7,9121.0,13368.071429,4247.071429,4247.071429,46.56%,46.56%
3,9,公域搜索带券,1,9070.0,12996.700000,3926.7,3703.880000,40.84%,43.29%
4,9,其他渠道场景渠道。,3,8913.333333,12996.700000,4083.366667,3840.820000,43.09%,45.81%
5,9,小程序商家消息（服务提醒）,1,9000.0,12996.700000,3996.7,3996.700000,44.41%,44.41%
6,9,我的小程序入口,3,5575.0,8447.400000,2872.4,1618.151667,29.03%,51.52%
7,9,搜索-租手机,39,8722.358974,12730.074359,4007.715385,3664.256667,42.01%,45.95%
8,9,搜索-租苹果手机,18,9084.722222,13213.333333,4128.611111,3834.901667,42.21%,45.45%
9,9,搜索下挂-商品,119,8975.579832,13105.926891,4130.347059,3731.214370,41.57%,46.02%


In [172]:
dfck_zsb=dfck2.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,归属渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,京享租,448,8755.154018,12674.731250,3919.577232,4270.611473,48.78%,44.77%
1,9,付费流量(通过商家数字推广平台，灯火等投放的广告),2,9820.0,14296.500000,4476.5,4137.050000,42.13%,45.59%
2,9,八派信息,7,9121.0,13368.071429,4247.071429,4247.071429,46.56%,46.56%
3,9,其他渠道场景渠道。,3,8913.333333,12996.700000,4083.366667,3840.820000,43.09%,45.81%
4,9,小程序商家消息（服务提醒）,1,9000.0,12996.700000,3996.7,3996.700000,44.41%,44.41%
5,9,我的小程序入口,2,3997.5,6172.750000,2175.25,1184.202500,29.62%,54.42%
6,9,搜索渠道,521,8863.527831,12936.136468,4072.608637,3732.125317,42.11%,45.95%
7,9,支付宝客户端首页,12,7698.0,11144.483333,3446.483333,3084.403750,40.07%,44.77%
8,9,支付宝直播,4,9305.0,13646.600000,4341.6,2776.955000,29.84%,46.66%
9,9,支群-社群专享,1,9000.0,12996.700000,3996.7,3731.880000,41.47%,44.41%


In [173]:
dfck_zsb =dfck2[dfck2["归属渠道"]=="搜索渠道"].groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,9,521,8863.527831,12936.136468,4072.608637,3732.125317,42.11%,45.95%


# 商家数据

In [174]:
# 去重订单可能会与日报数据存在些许出路，这是因为该用户在多个商家或渠道下过订单，去重的时候仅会保留一个订单
merchant_list = ['澄心优租', '优优2店', '北京海鸟窝科技有限公司', '租着用电脑数码', '趣智数码', '汇客好租', '乙辉数码','小蚂蚁租机', '兴鑫兴通讯', '呱子笔记本电脑', '南京聚格网络科技', '人人享租', '喜卓灵租机', '喜卓灵新租机', '星晟数码', '蘑菇时间']
df_merchant = df_contain[df_contain.商家.isin(merchant_list)]
df_merchant.loc[:, '是否进件'] = np.where(df_merchant.进件=='进件', 1, 0)
df_merchant.loc[:, '出库'] = np.where(df_merchant.审核状态=='出库', 1, 0)
df_merchant_group = df_merchant.groupby(['月份', '商家']).agg({'单号': 'count', '是否进件': 'sum', '出库': 'sum', '总租金': 'mean', '买断金额': 'mean'}).rename(columns={'单号': '去重订单数', '是否进件': '进件数', '总租金': '租金平均价', '买断金额': '买断平均价'})
df_merchant_group.loc[:, '进件出库率'] = (df_merchant_group.出库/df_merchant_group.进件数).map(lambda x: format(x,".2%"))
df_merchant_group = df_merchant_group.reset_index()
df_merchant_group[['月份', '商家', '去重订单数', '进件数', '出库', '进件出库率', '租金平均价', '买断平均价']]

,月份,商家,去重订单数,进件数,出库,进件出库率,租金平均价,买断平均价
0,9,人人享租,903,901,19,2.11%,7441.800532,13297.827464
1,9,北京海鸟窝科技有限公司,21,7,0,0.00%,665.842857,3247.619048
2,9,南京聚格网络科技,1,1,0,0.00%,180.000000,6500.000000
3,9,喜卓灵新租机,2645,2644,58,2.19%,7816.181009,13289.683100
4,9,星晟数码,477,156,3,1.92%,4213.139706,8600.663082
5,9,汇客好租,785,414,18,4.35%,5441.170178,10181.146497
6,9,澄心优租,25,6,0,0.00%,3651.120000,6113.600000
7,9,租着用电脑数码,140,23,1,4.35%,2028.563429,4613.585714
8,9,蘑菇时间,61,16,3,18.75%,1049.952787,4499.883607
9,9,趣智数码,47,18,2,11.11%,386.127660,5195.914894


In [175]:
# df_merchant_xcsm = df_contain[df_contain['商家']=='星晟数码']
# df_merchant_xcsm.loc[:, '是否进件'] = np.where(df_merchant_xcsm.进件=='进件', 1, 0)
# df_merchant_xcsm.loc[:, '出库'] = np.where(df_merchant_xcsm.审核状态=='出库', 1, 0)
# df_group = df_merchant_xcsm.groupby('下单日期').agg({'单号': 'count', '是否进件': 'sum', '出库': 'sum', '总租金': 'mean', '买断金额': 'mean'}).rename(columns={'单号': '去重订单数', '是否进件': '进件数', '总租金': '租金平均价', '买断金额': '买断平均价'})
# df_group

,去重订单数,进件数,出库,租金平均价,买断平均价
下单日期,,,,,
2025-09-01,134,46,2,4267.155224,8757.125970
2025-09-02,118,40,1,4108.804831,8495.678390
2025-09-03,104,42,0,4297.115288,8669.700577
2025-09-04,43,14,0,4196.602326,8619.439535
2025-09-05,8,0,0,3178.000000,7550.000000
2025-09-06,4,1,0,2978.100000,7075.100000
2025-09-15,66,13,0,4368.781515,8569.487879


In [176]:
# df1 = pd.read_excel('F:/myfile/p站数据/当月数据/0801-0815.xlsx')
# # df0 =df1.copy()
# df1.shape

In [177]:
# df0_xcsm = df1[df1['商家']=='星晟数码']
# df0_xcsm["下单日期"]=df0_xcsm["下单时间"].dt.date
# df0_xcsm["下单日期"]=pd.to_datetime(df0_xcsm["下单日期"],errors="coerce")
# df0_xcsm["下单日期"]
# df0_xcsm["月份"]=df0_xcsm["下单日期"].dt.month
# #删除重复单号
# df0_xcsm.drop_duplicates(subset=["单号"],inplace=True)
# #删除身份证空值行
# df0_xcsm.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
# #去刷单订单
# df0_xcsm.drop(df0_xcsm[df0_xcsm['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单秘密计划"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单曙光计划"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="线下门店3个月试行"].index,inplace=True)
# #删除重复单号
# df0.drop_duplicates(subset=["单号"],inplace=True)
# #删除订单状态空值行
# df0_xcsm.dropna(subset=["订单状态"],axis=0,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)
# df0_xcsm.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)
# df0_xcsm.单号.count()

In [178]:
# df_xcsm_list = df0_xcsm1.单号.to_list()
# len(df_xcsm_list)

In [179]:
# df0 =df1.copy()
# df0["下单日期"]=df0["下单时间"].dt.date
# df0["下单日期"]=pd.to_datetime(df0["下单日期"],errors="coerce")
# df0["下单日期"]
# df0["月份"]=df0["下单日期"].dt.month


In [180]:
# #删除重复单号
# df0.drop_duplicates(subset=["单号"],inplace=True)
# #删除身份证空值行
# df0.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
# #去刷单订单
# df0.drop(df0[df0['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单秘密计划"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单曙光计划"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="线下门店3个月试行"].index,inplace=True)
# #删除订单状态空值行
# df0.dropna(subset=["订单状态"],axis=0,inplace=True)

In [181]:
# df0.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)


In [182]:
# df0.drop(df0[df0['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)



# 机型均值

In [183]:
# # 商品押金  签约价   采购价   
dfck2["型号内存"]=dfck2["商品型号"]+"_"+dfck2["内存_x"]
dfck2.groupby(['型号内存']).agg({'商品押金':'mean','买断金额':'mean','成本':'mean'})

,商品押金,买断金额,成本
型号内存,,,
REDMI K80_16GB+1TB,3599.0,4642.700000,2952.0
iPad mini 7_256GB WiFi版,4499.0,5847.800000,4192.0
iPhone 14 Pro_256GB,7564.2,7564.200000,4135.0
iPhone 16e_128GB,4499.0,5847.800000,4089.0
iPhone15Pro_128GB,7999.0,8478.900000,7555.0
iPhone15Pro_256G,8999.0,9449.000000,5165.0
iPhone16Plus_128GB,6999.0,9097.300000,5934.0
iPhone16Plus_256GB,7999.0,10343.766667,7116.333333
iPhone16ProMax_256G,9999.0,12947.700000,9010.0


In [184]:
# df_zulin = dfck2.groupby(['型号内存']).agg({'订单号':'count', '商品押金':'mean','买断金额':'mean','成本':'mean'})
# df_zulin.to_excel(f'F:/宏生/{excel_date[-1]}宏生转化金额.xlsx')